# Cadence Conversion

## Input formats

nx(<list>)+mx(<list>)+...

where
n,m are positive integers
<lists> are one or more rep counts.
    
*Example*
    
1x(1,2,3) should become [1,2,3]

### Case 0:  Setup

In [1]:
import numpy as np
import pandas as pd
import os
from pathlib import Path
import re
from sqlalchemy import (create_engine, text)


PGURI = os.environ['PG_AZURE_URI']
PGPW = os.environ['PG_AZURE_PW']


datasets = Path('/Users/edmundlskoviak/iCloud/Data Sets/')

## Define Functions


In [3]:

def tokenize(inStr):
    if inStr.find('+') > -1:
        return inStr.split('+')
    else:
        return [inStr]

def parseOperators(inStr):
    outList = []
    mode = 'tile'
    if re.match("BW", inStr) or re.match("max", inStr):
        return inStr

    if inStr.count('r') == 1:
        mode = 'repeat'
        inStr = inStr.replace('r','')

    inStr.replace('*','x')
    
    if inStr.count('x') == 0:
        # basic Set
        mult = 1
        set=inStr
    else:
        (mult, set) = inStr.split('x')

    # need to check to see if set has an operator embedded
    if set.count('-->') == 1:
        start, stop = set.split('-->')
        start = int(re.sub('\(','',start))
        stop = int(re.sub('\)','',stop))
        if stop < start:
            step = -1
            stop -= 1
        else:
            step = 1
            stop += 1
        set = np.arange(start, stop, step)
    else:
        # otherwise, evaluate set
        try:
            set = list(eval(set))
        except TypeError as te:
            set = list(eval(set+','))
        except NameError as ne:
            return set
    if mode == 'tile':
        outList= np.tile(set, int(mult))
    elif mode == 'repeat':
        outList = np.repeat(set, int(mult))

    return outList


In [5]:
print(os.environ['PG_AZURE_URI'])

postgresql://citus@c.homer.postgres.database.azure.com:5432/citus?sslmode=require&sslrootcert=/Users/edmundlskoviak/.ssh/DigiCertGlobalRootG2.crt.pem


In [3]:
engine = create_engine(os.environ['PG_AZURE_URI'])
sets_qry_stmt = """
SELECT s.id
    ,date
    ,exercise_name
    ,cadence
    ,weights
    ,weight_unit
    ,notes
FROM sets s
JOIN exercises e ON e.id = s.exercise_id
ORDER BY date; 
"""
with engine.connect() as conn:
    #results = conn.execute(text(sets_qry_stmt))
    #print(results)
    raw_sets_df = pd.read_sql_query(text(sets_qry_stmt), conn)

OperationalError: (psycopg2.OperationalError) connection to server at "c.homer.postgres.database.azure.com" (20.109.47.254), port 5432 failed: fe_sendauth: no password supplied

(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [45]:
for row in raw_sets_df.itertuples():
        print(f'Id: {row[1]}', end='')
        print(f' Cadence: {(row[4])} Weights: {(row[5])}')
        #print(f' Volume: {sum(token * row[5])}')
    


Id: 6230 Cadence: 4,6,10,6 Weights: [95,100,135,135]
Id: 6229 Cadence: 3,4,7,4 Weights: 60,65,65,65
Id: 6228 Cadence: [6,5,4,3,3] Weights: 45,45,50,60,65
Id: 6231 Cadence: (4,6,10,6) Weights: 20,20,20,20
Id: 6234 Cadence: 3,4,7,4 Weights: [135,155,185,205]
Id: 6233 Cadence: 6,5,4,3,3 Weights: [135,145,155,165,170]
Id: 6232 Cadence: 6,5,4,3,3 Weights: 75,95,100,105,105
Id: 6235 Cadence: 4,6,10,6 Weights: 95,95,95,95
Id: 6239 Cadence: (4,6,10,6) Weights: 22,22,22,22
Id: 6237 Cadence: 3,4,7,4 Weights: 55,55,55,55
Id: 6236 Cadence: 6,5,4,3,3 Weights: [185,225,250,270,275]
Id: 6238 Cadence: 4,6,10,6 Weights: 10,10,10,10
Id: 6242 Cadence: 3,4,7,3,4,6 Weights: 95,95,95,95,95,95
Id: 6241 Cadence: 1,1 Weights: 28,28
Id: 6240 Cadence: 1,3,1,1,1 Weights: 60,60,60,70,70
Id: 6243 Cadence: 3,4,7,3,4,6 Weights: [155,155,155,155,155,155]
Id: 6244 Cadence: (4,6,10,4,6,10,4,6) Weights: 22,22,22,22,22,22,22,22
Id: 6245 Cadence: 1,3,1,1,1 Weights: [155,160,160,175,185]
Id: 6246 Cadence: 1,3,1,1,1 Weights:

# New Approach

The csv data referenced below is an extract from the online database.  Need to parse it and create a table to be scanned into the R processor

In [47]:
df = pd.read_csv( datasets / 'sets.csv')

### Case 1:  nx(<list>)

In [17]:
for cadence in raw_sets_df['cadence']:
    listFinal = []
    for token in tokenize(cadence):
        listFinal.extend(parseOperators(token))
    print(f"In: {cadence}\t Out: {listFinal}")

In: 4,6,10,6	 Out: [4, 6, 10, 6]
In: 3,4,7,4	 Out: [3, 4, 7, 4]
In: [6,5,4,3,3]	 Out: [6, 5, 4, 3, 3]
In: (4,6,10,6)	 Out: [4, 6, 10, 6]
In: 3,4,7,4	 Out: [3, 4, 7, 4]
In: 6,5,4,3,3	 Out: [6, 5, 4, 3, 3]
In: 6,5,4,3,3	 Out: [6, 5, 4, 3, 3]
In: 4,6,10,6	 Out: [4, 6, 10, 6]
In: (4,6,10,6)	 Out: [4, 6, 10, 6]
In: 3,4,7,4	 Out: [3, 4, 7, 4]
In: 6,5,4,3,3	 Out: [6, 5, 4, 3, 3]
In: 4,6,10,6	 Out: [4, 6, 10, 6]
In: 3,4,7,3,4,6	 Out: [3, 4, 7, 3, 4, 6]
In: 1,1	 Out: [1, 1]
In: 1,3,1,1,1	 Out: [1, 3, 1, 1, 1]
In: 3,4,7,3,4,6	 Out: [3, 4, 7, 3, 4, 6]
In: (4,6,10,4,6,10,4,6)	 Out: [4, 6, 10, 4, 6, 10, 4, 6]
In: 1,3,1,1,1	 Out: [1, 3, 1, 1, 1]
In: 1,3,1,1,1	 Out: [1, 3, 1, 1, 1]
In: 3,4,7,3,4	 Out: [3, 4, 7, 3, 4]
In: 4,6,10,4,7,10,4,6	 Out: [4, 6, 10, 4, 7, 10, 4, 6]
In: 1,3,1,1	 Out: [1, 3, 1, 1]
In: 3,4,7,3,4,6	 Out: [3, 4, 7, 3, 4, 6]
In: 4,6,10,4,7,10,4,6	 Out: [4, 6, 10, 4, 7, 10, 4, 6]
In: 1,3,3,1,2	 Out: [1, 3, 3, 1, 2]
In: 1,3,3,1,2	 Out: [1, 3, 3, 1, 2]
In: 3,5	 Out: [3, 5]
In: 4,6,10,7	

In [18]:
for weight in raw_sets_df['weights']:
    listFinal = []
    for token in tokenize(weight):
        listFinal.extend(parseOperators(token))
    print(f"In: {weight}\tOut: {listFinal}")

In: [95,100,135,135]	Out: [95, 100, 135, 135]
In: 60,65,65,65	Out: [60, 65, 65, 65]
In: 45,45,50,60,65	Out: [45, 45, 50, 60, 65]
In: 20,20,20,20	Out: [20, 20, 20, 20]
In: [135,155,185,205]	Out: [135, 155, 185, 205]
In: [135,145,155,165,170]	Out: [135, 145, 155, 165, 170]
In: 75,95,100,105,105	Out: [75, 95, 100, 105, 105]
In: 95,95,95,95	Out: [95, 95, 95, 95]
In: 22,22,22,22	Out: [22, 22, 22, 22]
In: 55,55,55,55	Out: [55, 55, 55, 55]
In: [185,225,250,270,275]	Out: [185, 225, 250, 270, 275]
In: 10,10,10,10	Out: [10, 10, 10, 10]
In: 95,95,95,95,95,95	Out: [95, 95, 95, 95, 95, 95]
In: 28,28	Out: [28, 28]
In: 60,60,60,70,70	Out: [60, 60, 60, 70, 70]
In: [155,155,155,155,155,155]	Out: [155, 155, 155, 155, 155, 155]
In: 22,22,22,22,22,22,22,22	Out: [22, 22, 22, 22, 22, 22, 22, 22]
In: [155,160,160,175,185]	Out: [155, 160, 160, 175, 185]
In: [100,105,110,110,110]	Out: [100, 105, 110, 110, 110]
In: [225,225,225,225,225]	Out: [225, 225, 225, 225, 225]
In: 100	Out: [100]
In: [225,245,245,280]	Out

SyntaxError: invalid syntax (<string>, line 0)

In [ ]:
# superceded
cadences = df['cadence'].to_numpy()

for indata in cadences:
    print(f"{indata}:")
    listCadence = parseSeries(indata)
    print(listCadence)

    for cadence in listCadence:
        print(cadence)
        print(f"{getSeries(cadence)}")

#    print(listCadence)
#    if indata.find('+') > -1:
#        print(indata, indata.split('+'))
#    else:  
#        print(f"In: {indata} out: {getSeries(indata)}")

In [ ]:
row, col = df.shape
print (row, col)

In [ ]:
listL = []
rem = ''
start = 0
while len(helper) > 0:
    stop = helper.find('x')
    if stop == -1:
        listL.append(helper[start:])
        helper = ''
    else:
        listL.append(helper[start:stop-2])
        start = stop-1
        helper = helper[start:]
        break

print(listL, helper)

In [ ]:
def tokenize(inStr):
    if inStr.find('+') > -1:
        return inStr.split('+')
    else:
        return [inStr]

def parseOperators(inStr):
    outList = []
    inStr.replace('*','+')
    if inStr.count('x') == 0:
        # basic Set
        outList=list(eval(inStr))
    
    if inStr.count('x') == 1:
        # in  n x (...)[t | r]  default to t
        mode = 'tile'
        if inStr.count('r') == 1:
            mode = 'repeat'
            inStr = inStr.replace('r','')
        (mult, set) = inStr.split('x')
        if mode == 'tile':
            outList= np.tile(list(eval(set)), int(mult))
        elif mode == 'repeat':
            outList = np.repeat(list(eval(set)), int(mult))

    return outList
 
helper = '2x(5,3,2)+2x(1,3)'
#print(tokenize(helper))
#print(parseOperators(tokenize(helper)[0]))
listFinal = []
for token in tokenize(helper):
    listFinal.extend(parseOperators(token))
    print(parseOperators(token))

print(listFinal)

In [ ]:
import re
i = '(1,)'
re.sub('\(','',i)
#i = i.replace('()','')
#print(int('(1'))

In [ ]:
i = '(1,2,3)'
eval(i)
list(eval(i))
y = (1,2,4)
list(y)